In [81]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

import time

In [82]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.3.1+cu121
True


In [83]:
model = YOLO('../best.pt')
imgs = []
filenames = sorted(os.listdir('imgs_test'), key=lambda x: int(x.split('.')[0]))

In [84]:
for filename in filenames:
    with open(f'imgs_test/{filename}', 'rb') as f:
        imgs.append(f.read())

In [85]:
def bytes_to_image(byte_data):
    np_arr = np.frombuffer(byte_data, np.uint8)
    image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    print(type(image))
    return image

In [86]:
boundary_y = 50
entry_count = 0
exit_count = 0
confidence_threshold = 0.9
previous_positions = {}

In [87]:
def process_and_track(frames):
    global entry_count, exit_count, previous_positions
    for byte_data in frames:
        # Convert byte data to image
        frame = bytes_to_image(byte_data)
        # Ensure the frame was converted successfully
        if frame is None:
            continue
        # Track objects in the frame
        results = model.track(source=frame,device=0) # usando a gpu em 0
        
        # Process the tracking results (e.g., draw bounding boxes)
        current_positions = {}
        
        for result in results:
            for box in result.boxes:
                confidence = box.conf[0].item()

                if confidence < confidence_threshold:
                    continue

                x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
                #mudança try catch
                try:
                    object_id = int(box.id[0])  # Assuming 'id' contains the unique ID
                except:
                    object_id = 0
                
                # Calculate the center of the bounding box
                center_y = (y1 + y2) // 2
                
                # Store the current position
                current_positions[object_id] = center_y
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
                #cv2.putText(frame, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # Check for entries and exits
        for object_id, center_y in current_positions.items():
            if object_id in previous_positions:
                previous_y = previous_positions[object_id]
                
                # Check if the object has crossed the boundary
                if previous_y <= boundary_y < center_y:
                    entry_count += 1
                elif previous_y >= boundary_y > center_y:
                    exit_count += 1
        
        # Update previous positions
        previous_positions = current_positions
        
        # Draw the boundary line
        cv2.line(frame, (0, boundary_y), (frame.shape[1], boundary_y), (0, 255, 0), 2)
        
        resized_frame = cv2.resize(frame, (500, 500))

        cv2.putText(resized_frame, f'Entries: {entry_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
        cv2.putText(resized_frame, f'Exits: {exit_count}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
    
        # Display the frame (optional, for visualization purposes)
        cv2.imshow('Tracked Frame', resized_frame)
        #time.sleep(0.25)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [88]:
process_and_track(imgs)
cv2.destroyAllWindows()

<class 'numpy.ndarray'>

0: 640x640 1 person, 79.8ms
Speed: 4.0ms preprocess, 79.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 2 persons, 80.8ms
Speed: 5.0ms preprocess, 80.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 1 person, 75.8ms
Speed: 3.0ms preprocess, 75.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 1 person, 73.8ms
Speed: 4.0ms preprocess, 73.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 1 person, 69.8ms
Speed: 3.0ms preprocess, 69.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 1 person, 73.8ms
Speed: 6.0ms preprocess, 73.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
<class 'numpy.ndarray'>

0: 640x640 2 persons, 72.8ms
Speed: 3.0ms preprocess, 72.8ms inference, 3.